In [ ]:
# %pip install sentence_transformers

# import sentence_transformers
# from sentence_transformers import SentenceTransformer

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report
import numpy as np

# 1. Cargar el dataset
df = pd.read_csv('../../data/processed/books.csv')

# 2. Combinar título y blurb como entrada textual
df['text'] = df['book_title'].fillna('') + '. ' + df['blurb'].fillna('')

# 3. Procesar los tags: convertirlos en listas
df['tags'] = df['tags'].apply(lambda x: [t.strip() for t in str(x).split(',')])

# 4. Binarización multilabel
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['tags'])

# 5. Generar embeddings con Sentence-BERT
model = SentenceTransformer('all-MiniLM-L6-v2')
X_embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# 6. Entrenar modelo multilabel
clf = MultiOutputClassifier(LogisticRegression(max_iter=1000))
clf.fit(X_embeddings, Y)

# 7. Función para predecir tags de nuevos libros
def predict_tags(titles, blurbs):
    texts = [t + '. ' + b for t, b in zip(titles, blurbs)]
    X_test = model.encode(texts)
    preds = clf.predict(X_test)
    tag_lists = mlb.inverse_transform(preds)
    return tag_lists

# Ejemplo de predicción
new_titles = ["La conspiración del universo"]
new_blurbs = ["Una historia que entrelaza ciencia, fe y filosofía para revelar los secretos de la creación."]
predicted_tags = predict_tags(new_titles, new_blurbs)
print(predicted_tags)


Batches:   0%|          | 0/37 [00:00<?, ?it/s]

[('to-read',)]
